# Libraries

In [ ]:
!pip install gurobipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from itertools import chain, combinations, permutations, product
import time
from tqdm import tqdm
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
from copy import deepcopy

In [ ]:
WLSACCESSID = '< Copy your WLSACCESSID here >'
WLSSECRET = '< Copy your WLSSECRET here >'
LICENSEID = '< Copy your LICENSEID here >'

In [ ]:
params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID,
}
env = gp.Env(params=params)

# Data

In [ ]:
widths = [4,1,2,3,2,6,10]
heights =[3,1,1,2,4,2,4]
quants = [4,10,8,5,2,3,2]

In [ ]:
df = pd.DataFrame()
df['quantity'] = quants
df['heights'] = heights
df['widths'] = widths
df

In [ ]:
WIDTHS = []
HEIGHTS = []
for q,w,h in zip(quants,widths,heights):
  for x in range(q):
    HEIGHTS.append(w)
    WIDTHS.append(h)

data_df = pd.DataFrame()
data_df['HEIGHTS'] = HEIGHTS
data_df['WIDTHS'] = WIDTHS

In [ ]:
data_df

In [ ]:
N = len(data_df)
N

In [ ]:
top = max(data_df['HEIGHTS'].sum(),data_df['WIDTHS'].sum())

In [ ]:
M = top

# Minimize total Area needed: Rotations allowed

The idea is to minimize the total area needed to store all the objects. So there is not going to be a fixed rectangle

Formulation

* Variables

    1. $x_i, y_i$: Coordinates of the bottom-left corner of rectangle $i$
    2. $b_{ij}^k$: Binary variables to handle the "or" conditions for non-overlapping constraints ($k \in \{1, 2, 3, 4\}$)
    3. $\overline{x}$, $\overline{y}$: Maximum **X** and **Y** values
    4. $r_i$: Binary variable that handles if item $i$ is rotated or not

* Objective Function
$$
\text{Minimize} \quad \overline{x} \cdot \overline{y}
$$

* Constraints
\begin{align}
    & \overline{x} \geq x_i + w_ir_i + (1-r_i)h_i \quad \forall i \\
    & \overline{y} \geq y_i + h_ir_i + (1-r_i)w_i \quad \forall i \\
    & x_i + w_ir_i + (1-r_i)h_i  \leq x_j + M \cdot (1- b_{ij}^1) \quad \forall i \neq j\\
    & x_j + w_jr_j + (1-r_j)h_j  \leq x_i + M \cdot(1- b_{ij}^2) \quad \forall i \neq j\\
    & y_i + h_ir_i + (1-r_i)w_i  \leq y_j + M \cdot(1- b_{ij}^3) \quad \forall i \neq j\\
    & y_j + h_jr_j + (1-r_j)w_j  \leq y_i + M \cdot(1- b_{ij}^4) \quad \forall i \neq j\\
    & \sum_{k=1}^{4}b_{ij}^k \geq 1 \quad \forall i \neq j \\
    & z_i \in \{0, 1\} \quad \forall i \\
    & b_{ij}^k \in \{0, 1\} \quad \forall i \neq j, \quad \forall k \in \{1, 2, 3 , 4\}
\end{align}

In [ ]:
model = gp.Model("Assortment",env=env)

## Variables

In [ ]:
I = range(N)

In [ ]:
K = range(4)

In [ ]:
x = model.addVars(I,lb = 0,ub = top,vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(I,lb = 0,ub = top,vtype=GRB.CONTINUOUS, name="y")

In [ ]:
R = model.addVars(I,vtype=GRB.BINARY,name = 'R')

In [ ]:
X = model.addVar(lb=0,ub = top,vtype = GRB.CONTINUOUS,name = "X")
Y = model.addVar(lb=0,ub = top,vtype = GRB.CONTINUOUS,name = "Y")

In [ ]:
b_vars = [(i,j,k) for i in I for j in I if j!=i for k in K]

In [ ]:
B = model.addVars(b_vars,vtype = GRB.BINARY,name = "B")

## Objective function

In [ ]:
model.setObjective(X*Y,GRB.MINIMIZE);

## Constraints

In [ ]:
for i in I:

  model.addConstr(X >= x[i] + WIDTHS[i]*R[i] + (1-R[i])*HEIGHTS[i])
  model.addConstr(Y >= y[i] + HEIGHTS[i]*R[i] + (1-R[i])*WIDTHS[i])

In [ ]:
for i in I:
  for j in I:
    if i == j:
      continue
    else:
      model.addConstr(x[i] + WIDTHS[i]*R[i] + (1-R[i])*HEIGHTS[i] <= x[j] + M*(1-B[i,j,0]))
      model.addConstr(x[j] + WIDTHS[j]*R[j] + (1-R[j])*HEIGHTS[j] <= x[i] + M*(1-B[i,j,1]))

      model.addConstr(y[i] + HEIGHTS[i]*R[i] + (1-R[i])*WIDTHS[i] <= y[j] + M*(1-B[i,j,2]))
      model.addConstr(y[j] + HEIGHTS[j]*R[j] + (1-R[j])*WIDTHS[j] <= y[i] + M*(1-B[i,j,3]))

      model.addConstr(B[i,j,0] + B[i,j,1] + B[i,j,2] + B[i,j,3] >= 1)

# Solving the model

In [ ]:
tl = 600
mip_gap = 0.05

In [ ]:
model.setParam('TimeLimit', tl)
model.setParam('MIPGap', mip_gap)
model.optimize()

# Extracting the solution

In [ ]:
all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

In [ ]:
obj = round(model.getObjective().getValue(),0)

In [ ]:
total_X = int(round((X.x),0))
total_Y = int(round((Y.x),0))

In [ ]:
output_list = []
for i in I:

  print(f"item {i} x:{x[i].x}, y:{y[i].x}, Rotated:{R[i].x <= 0.01}")
  row = {'item':i,'x':round(x[i].x,2),'y':round(y[i].x,2),'Rotated':R[i].x <= 0.01}
  output_list.append(row)

In [ ]:
output_df = pd.DataFrame(output_list)
output_df.to_csv("output_solution.csv")

In [ ]:
fig, ax = plt.subplots()

for item in I:

  coords = (x[item].x,y[item].x)

  if R[item].x <= 0.01:
    wid = HEIGHTS[item]
    hig = WIDTHS[item]
  else:
    wid = WIDTHS[item]
    hig = HEIGHTS[item]

  ax.add_patch(Rectangle(coords, wid, hig,
            edgecolor = 'black',
            facecolor = "Grey",
            fill = True,
            alpha = 0.5,
            lw=2))
ax. set_xlim(0, total_X )
ax. set_ylim(0, total_Y )

ax.set_xticks(range(total_X+1))
ax.set_yticks(range(total_Y+1))
ax.grid()
ax.set_title(f" Total area {total_X} x {total_Y} = {int(obj)}")

plt.show()

# Heuristic Comparisson

In [ ]:
class Rectangle_class:
    def __init__(self, width, height,index):
        self.width = width
        self.height = height
        self.x = 0
        self.y = 0
        self.index = index

## FFDH: First Fit Decreasing Height (FFDH)

This approach sorts rectangles by height and then places each rectangle in the first available position that fits.

In [ ]:
def ffdh(rectangles):
    # Sort rectangles by height in descending order
    rectangles.sort(key=lambda rect: rect.height, reverse=True)

    # Initialize variables to keep track of the positions
    current_y = 0
    current_x = 0
    row_height = 0
    total_width = 0

    for rect in rectangles:
        if current_x + rect.width > total_width:
            total_width = current_x + rect.width
        # If rectangle fits in the current row
        if current_x + rect.width <= total_width:
            rect.x = current_x
            rect.y = current_y
            current_x += rect.width
            row_height = max(row_height, rect.height)
        else:
            # Move to the next row
            current_y += row_height
            rect.x = 0
            rect.y = current_y
            current_x = rect.width
            row_height = rect.height

    total_height = current_y + row_height
    return rectangles, total_width, total_height

In [ ]:
rectangles = []

In [ ]:
for i in range(len(data_df)):

  h,w = data_df.iloc[i,0], data_df.iloc[i,1]
  REC = Rectangle_class(w,h,i)
  rectangles.append(REC)

In [ ]:
packed_rectangles, total_width, total_height = ffdh(rectangles)

In [ ]:
total_X = total_width
total_Y = total_height

In [ ]:
obj = total_X*total_Y

In [ ]:
for rect in packed_rectangles:

  print(rect.index,rect.x,rect.y,rect.width,rect.height)
print(f"Total Area {total_width} x {total_height} = {obj}")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

for rect in packed_rectangles:

  coords = (rect.x,rect.y)
  wid,hig = rect.width,rect.height

  ax.add_patch(Rectangle(coords, wid, hig,
            edgecolor = 'black',
            facecolor = "Grey",
            fill = True,
            alpha = 0.5,
            lw=2))
ax. set_xlim(0, total_X )
ax. set_ylim(0, total_Y )

ax.set_xticks(range(total_X+1))
ax.set_yticks(range(total_Y+1))
ax.grid()
ax.set_title(f" Total area {total_X} x {total_Y} = {int(obj)}")
plt.xticks(rotation=30)
plt.show()

## Shelves Heuristic

In [ ]:
def shelf_heuristic(rectangles, max_width):
    # Sort rectangles by height in descending order
    rectangles.sort(key=lambda rect: rect.height, reverse=True)

    current_x = 0
    current_y = 0
    shelf_height = 0

    for rect in rectangles:
        if current_x + rect.width > max_width:
            # Move to a new shelf
            current_x = 0
            current_y += shelf_height
            shelf_height = 0

        # Place the rectangle
        rect.x = current_x
        rect.y = current_y
        current_x += rect.width
        shelf_height = max(shelf_height, rect.height)

    # total_width = max_width
    # total_height = current_y + shelf_height

    total_width = max([rec.x + rec.width for rec in rectangles])
    total_height = max([rec.y + rec.height for rec in rectangles])

    return rectangles, total_width, total_height

In [ ]:
container_width = 20

In [ ]:
packed_rectangles,total_width, total_height = shelf_heuristic(rectangles, container_width)

In [ ]:
total_X = total_width
total_Y = total_height

In [ ]:
obj = total_X*total_Y

In [ ]:
for rect in packed_rectangles:

  print(rect.index,rect.x,rect.y,rect.width,rect.height)
print(f"Total Area {total_width} x {total_height} = {obj}")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

for rect in packed_rectangles:

  coords = (rect.x,rect.y)
  wid,hig = rect.width,rect.height

  ax.add_patch(Rectangle(coords, wid, hig,
            edgecolor = 'black',
            facecolor = "Grey",
            fill = True,
            alpha = 0.5,
            lw=2))
ax. set_xlim(0, total_X )
ax. set_ylim(0, total_Y )

ax.set_xticks(range(total_X+1))
ax.set_yticks(range(total_Y+1))
ax.grid()
ax.set_title(f" Total area {total_X} x {total_Y} = {int(obj)}")
plt.xticks(rotation=30)
plt.show()

### Optimization problem with warm start from heuristic

In [ ]:
heuristic_dict = dict()
for rect in packed_rectangles:
  index = rect.index
  heuristic_dict[index] = rect


b_values = dict()
for i in I:
  for j in I:
    if i == j:
      continue
    else:
      rect_i = heuristic_dict[i]
      rect_j = heuristic_dict[j]
      b_values[(i,j,0)] = 0
      b_values[(i,j,1)] = 0
      b_values[(i,j,2)] = 0
      b_values[(i,j,3)] = 0

      #print(f"({i},{j}) --> {rect_i.x} + {rect_i.width} <= {rect_j.x}")
      if rect_i.x + rect_i.width <= rect_j.x:
        b_values[(i,j,0)] = 1

      #print(f"({i},{j}) --> {rect_j.x} + {rect_j.width} <= {rect_i.x}")
      if rect_j.x + rect_j.width <= rect_i.x:
        b_values[(i,j,1)] = 1

      #print(f"({i},{j}) --> {rect_i.y} + {rect_i.height} <= {rect_j.y}")
      if rect_i.y + rect_i.height <= rect_j.y:

        b_values[(i,j,2)] = 1

      #print(f"({i},{j}) --> {rect_j.y} + {rect_j.height} <= {rect_i.y}")
      if rect_j.y + rect_j.height <= rect_i.y:

        b_values[(i,j,3)] = 1
      #print(b_values[(i,j,0)],b_values[(i,j,1)],b_values[(i,j,2)],b_values[(i,j,3)])
      #print("-"*100)

In [ ]:
x_dict = dict()
y_dict = dict()
r_dict = dict()

for i in I:

  REC = heuristic_dict[i]

  x_dict[i] = REC.x
  y_dict[i] = REC.y
  r_dict[i] = 1

In [ ]:
model = gp.Model("Assortment_warm_start",env=env)

x = model.addVars(I,lb = 0,ub = top,vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(I,lb = 0,ub = top,vtype=GRB.CONTINUOUS, name="y")

R = model.addVars(I,vtype=GRB.BINARY,name = 'R')

X = model.addVar(lb=0,ub = top,vtype = GRB.CONTINUOUS,name = "X")
Y = model.addVar(lb=0,ub = top,vtype = GRB.CONTINUOUS,name = "Y")

b_vars = [(i,j,k) for i in I for j in I if j!=i for k in K]
B = model.addVars(b_vars,vtype = GRB.BINARY,name = "B")

model.setObjective(X*Y,GRB.MINIMIZE);

for i in I:

  model.addConstr(X >= x[i] + WIDTHS[i]*R[i] + (1-R[i])*HEIGHTS[i])
  model.addConstr(Y >= y[i] + HEIGHTS[i]*R[i] + (1-R[i])*WIDTHS[i])

for i in I:
  for j in I:
    if i == j:
      continue
    else:
      model.addConstr(x[i] + WIDTHS[i]*R[i] + (1-R[i])*HEIGHTS[i] <= x[j] + M*(1-B[i,j,0]))
      model.addConstr(x[j] + WIDTHS[j]*R[j] + (1-R[j])*HEIGHTS[j] <= x[i] + M*(1-B[i,j,1]))

      model.addConstr(y[i] + HEIGHTS[i]*R[i] + (1-R[i])*WIDTHS[i] <= y[j] + M*(1-B[i,j,2]))
      model.addConstr(y[j] + HEIGHTS[j]*R[j] + (1-R[j])*WIDTHS[j] <= y[i] + M*(1-B[i,j,3]))

      model.addConstr(B[i,j,0] + B[i,j,1] + B[i,j,2] + B[i,j,3] >= 1)

In [ ]:
for var in x:
  x[var].Start = x_dict[var]
  y[var].Start = y_dict[var]

  R[var].Start = r_dict[var]

In [ ]:
X.Start = total_X
Y.start = total_Y

In [ ]:
for var in B:
  B[var] = b_values[var]

In [ ]:
tl = 600
mip_gap = 0.05

In [ ]:
model.setParam('TimeLimit', tl)
model.setParam('MIPGap', mip_gap)
model.optimize()

## Second pass iterative improvement

In [ ]:
def optimize_placement(rectangles, L, H):
    for i in range(len(rectangles) - 1, 0, -1):
        rect = rectangles[i]
        for j in range(i):
            target = rectangles[j]
            if can_place_on_top(rect, target, rectangles):
                new_x = target.x
                new_y = target.y + target.height
                if new_y + rect.height <= H and new_x + rect.width <= L:
                    rect.x = new_x
                    rect.y = new_y
                    break

    total_width = max([rec.x + rec.width for rec in rectangles])
    total_height = max([rec.y + rec.height for rec in rectangles])

    return rectangles,  total_width, total_height

def can_place_on_top(rect, target, rectangles):
    new_x = target.x
    new_y = target.y + target.height
    for other in rectangles:
        if other != rect and overlap(rect, new_x, new_y, other):
            return False
    return True

def overlap(rect, new_x, new_y, other):
    if (new_x < other.x + other.width and new_x + rect.width > other.x and
        new_y < other.y + other.height and new_y + rect.height > other.y):
        return True
    return False

In [ ]:
def iterative_shelf(rectangles,verbose = False):

  W = sum([rec.width for rec in rectangles])
  w = max([rec.width for rec in rectangles])

  H = sum([rec.height for rec in rectangles])

  best_w = w
  best_area = H*W

  for w_i in range(w,W+1):
    opt_rectangles = deepcopy(rectangles)
    packed_rectangles,total_width, total_height = shelf_heuristic(opt_rectangles, w_i)

    if verbose:
      print("First pass")
      print(w_i,total_width, total_height,total_width*total_height)

    optimized_rectangles,total_width, total_height = optimize_placement(packed_rectangles,total_width, total_height)

    Area = total_width*total_height

    if Area < best_area:
      best_area = Area
      best_w = w_i

    if verbose:
      print("Second pass")
      print(w_i,total_width, total_height,total_width*total_height)
      print("-"*100)

  if verbose:
    print(f"Best width shelf to choose {best_w}, for an area of { best_area}")

  packed_rectangles,total_width, total_height = shelf_heuristic(rectangles, best_w)
  optimized_rectangles,total_width, total_height = optimize_placement(packed_rectangles,total_width, total_height)

  return optimized_rectangles,total_width, total_height

In [ ]:
packed_rectangles,total_width, total_height = iterative_shelf(rectangles,True)

In [ ]:
total_X = total_width
total_Y = total_height

In [ ]:
obj = total_X*total_Y

In [ ]:
for rect in packed_rectangles:

  print(rect.x,rect.y,rect.width,rect.height)
print(f"Total Area {total_width} x {total_height} = {obj}")

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

for rect in packed_rectangles:

  coords = (rect.x,rect.y)
  wid,hig = rect.width,rect.height

  ax.add_patch(Rectangle(coords, wid, hig,
            edgecolor = 'black',
            facecolor = "Grey",
            fill = True,
            alpha = 0.5,
            lw=2))
ax.set_xlim(0, total_X )
ax.set_ylim(0, total_Y )

ax.set_xticks(range(total_X+1))
ax.set_yticks(range(total_Y+1))
ax.grid()
ax.set_title(f" Total area {total_X} x {total_Y} = {int(obj)}")
plt.xticks(rotation=30)
plt.show()

In [ ]:
heuristic_dict = dict()
for rect in packed_rectangles:
  index = rect.index
  heuristic_dict[index] = rect


b_values = dict()
for i in I:
  for j in I:
    if i == j:
      continue
    else:
      rect_i = heuristic_dict[i]
      rect_j = heuristic_dict[j]
      b_values[(i,j,0)] = 0
      b_values[(i,j,1)] = 0
      b_values[(i,j,2)] = 0
      b_values[(i,j,3)] = 0

      #print(f"({i},{j}) --> {rect_i.x} + {rect_i.width} <= {rect_j.x}")
      if rect_i.x + rect_i.width <= rect_j.x:
        b_values[(i,j,0)] = 1

      #print(f"({i},{j}) --> {rect_j.x} + {rect_j.width} <= {rect_i.x}")
      if rect_j.x + rect_j.width <= rect_i.x:
        b_values[(i,j,1)] = 1

      #print(f"({i},{j}) --> {rect_i.y} + {rect_i.height} <= {rect_j.y}")
      if rect_i.y + rect_i.height <= rect_j.y:

        b_values[(i,j,2)] = 1

      #print(f"({i},{j}) --> {rect_j.y} + {rect_j.height} <= {rect_i.y}")
      if rect_j.y + rect_j.height <= rect_i.y:

        b_values[(i,j,3)] = 1
      #print(b_values[(i,j,0)],b_values[(i,j,1)],b_values[(i,j,2)],b_values[(i,j,3)])
      #print("-"*100)

In [ ]:
x_dict = dict()
y_dict = dict()
r_dict = dict()

for i in I:

  REC = heuristic_dict[i]

  x_dict[i] = REC.x
  y_dict[i] = REC.y
  r_dict[i] = 1

In [ ]:
model = gp.Model("Assortment_warm_start",env=env)

x = model.addVars(I,lb = 0,ub = top,vtype=GRB.CONTINUOUS, name="x")
y = model.addVars(I,lb = 0,ub = top,vtype=GRB.CONTINUOUS, name="y")

R = model.addVars(I,vtype=GRB.BINARY,name = 'R')

X = model.addVar(lb=0,ub = top,vtype = GRB.CONTINUOUS,name = "X")
Y = model.addVar(lb=0,ub = top,vtype = GRB.CONTINUOUS,name = "Y")

b_vars = [(i,j,k) for i in I for j in I if j!=i for k in K]
B = model.addVars(b_vars,vtype = GRB.BINARY,name = "B")

model.setObjective(X*Y,GRB.MINIMIZE);

for i in I:

  model.addConstr(X >= x[i] + WIDTHS[i]*R[i] + (1-R[i])*HEIGHTS[i])
  model.addConstr(Y >= y[i] + HEIGHTS[i]*R[i] + (1-R[i])*WIDTHS[i])

for i in I:
  for j in I:
    if i == j:
      continue
    else:
      model.addConstr(x[i] + WIDTHS[i]*R[i] + (1-R[i])*HEIGHTS[i] <= x[j] + M*(1-B[i,j,0]))
      model.addConstr(x[j] + WIDTHS[j]*R[j] + (1-R[j])*HEIGHTS[j] <= x[i] + M*(1-B[i,j,1]))

      model.addConstr(y[i] + HEIGHTS[i]*R[i] + (1-R[i])*WIDTHS[i] <= y[j] + M*(1-B[i,j,2]))
      model.addConstr(y[j] + HEIGHTS[j]*R[j] + (1-R[j])*WIDTHS[j] <= y[i] + M*(1-B[i,j,3]))

      model.addConstr(B[i,j,0] + B[i,j,1] + B[i,j,2] + B[i,j,3] >= 1)

In [ ]:
for var in x:
  x[var].Start = x_dict[var]
  y[var].Start = y_dict[var]

  R[var].Start = r_dict[var]

In [ ]:
X.Start = total_X
Y.start = total_Y

In [ ]:
for var in B:
  B[var] = b_values[var]

In [ ]:
tl = 600
mip_gap = 0.05

In [ ]:
model.setParam('TimeLimit', tl)
model.setParam('MIPGap', mip_gap)
model.optimize()